In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_vertical_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,8947.99,-0.06,0.00,0.05,0.00,-0.01,0.00,-0.00,-0.00
3,-0.06,2427.12,-0.02,0.01,-0.01,0.00,0.01,0.00,0.00
4,0.00,-0.02,662.36,0.01,0.00,0.00,-0.00,-0.00,0.00
5,0.05,0.01,0.01,169.69,-0.00,-0.00,0.00,0.00,0.00
6,0.00,-0.01,0.00,-0.00,40.76,-0.00,-0.00,0.00,0.00
7,-0.01,0.00,0.00,-0.00,-0.00,9.28,0.00,0.00,0.00
8,0.00,0.01,-0.00,0.00,-0.00,0.00,2.06,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.48,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.11


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,0.00000,0.00004,0.00001,-0.00003,0.00001,-0.00002,-0.00000
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00002,0.00007,0.00001,0.00001
4,0.00000,-0.00001,1.00000,0.00004,0.00000,0.00000,-0.00005,-0.00001,0.00002
5,0.00004,0.00001,0.00004,1.00000,-0.00000,-0.00003,0.00006,0.00001,0.00002
6,0.00001,-0.00003,0.00000,-0.00000,1.00000,-0.00002,-0.00000,0.00003,0.00001
7,-0.00003,0.00002,0.00000,-0.00003,-0.00002,1.00000,0.00002,0.00000,0.00001
8,0.00001,0.00007,-0.00005,0.00006,-0.00000,0.00002,1.00000,0.00004,0.00001
9,-0.00002,0.00001,-0.00001,0.00001,0.00003,0.00000,0.00004,1.00000,-0.00001
10,-0.00000,0.00001,0.00002,0.00002,0.00001,0.00001,0.00001,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.11566868285579182

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[8.90810910e+06 6.44663828e+05 4.70210115e+04 3.00105363e+03
 1.66294035e+02 8.08795432e+00 3.55994976e-01 1.47892258e-02
 1.88655535e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999288,-0.036502,-0.009192,-0.002336,-0.000562,-0.000129,-0.000028,-0.000007,-0.000002
1,0.036055,0.998398,-0.042372,-0.009981,-0.002390,-0.000542,-0.000118,-0.000028,-0.000007
2,0.010582,0.041485,0.997968,-0.046005,-0.010256,-0.002324,-0.000518,-0.000120,-0.000028
3,0.003140,0.011636,0.044914,0.997536,-0.051319,-0.010917,-0.002405,-0.000558,-0.000133
4,0.000904,0.003339,0.012252,0.049993,0.996761,-0.060336,-0.012556,-0.002892,-0.000692
5,0.000257,0.000943,0.003457,0.013532,0.058472,0.994994,-0.077912,-0.016905,-0.004029
6,0.000074,0.000272,0.001009,0.003921,0.016380,0.074288,0.989366,-0.120979,-0.026761
7,0.000025,0.000090,0.000332,0.001292,0.005382,0.023689,0.109703,0.964942,-0.237192
8,0.000012,0.000045,0.000167,0.000651,0.002712,0.011918,0.053728,0.232285,0.971086


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0007116093007837332,
 0.0016017751184440598,
 0.00203222735172659,
 0.0024637469933275202,
 0.0032394292761324106,
 0.005006186076736752,
 0.010634326580763465,
 0.03505786502120245,
 0.02891443474843003]